# Calculate the stock price change in three month after earning call release

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import pickle

path = '/Users/nemo/hku/mfin7036/nlp-ferrari'

In [88]:
prices = pd.read_csv(path + '/data/prices.csv', index_col='Date')

### exclude those earning call which is not valid

In [53]:
articles = pd.read_csv(path + '/data/locations.csv')
articles = articles[articles['Children'] == False]
articles = articles[articles['Company'] != 'WMT']
articles = articles[articles['Company'] != 'GOOGL']
articles.dropna(inplace=True)

articles.sort_values(by=['Company', 'Date'], inplace=True)
articles.reset_index(drop=True, inplace=True)

## get the begin date and the end date for a earning calls

In [111]:
articles['b_d'] = pd.to_datetime(articles['SpecificDate'].apply(lambda s:s[:10]))
articles['e_d'] = articles['b_d'] + pd.DateOffset(months=3)

0      2011-01-18
1      2011-04-20
2      2011-07-19
3      2011-10-18
4      2012-01-24
          ...    
2038   2021-02-02
2039   2021-04-30
2040   2021-07-30
2041   2021-10-29
2042   2022-02-01
Name: b_d, Length: 2043, dtype: datetime64[ns]

In [124]:
def cal_price_change(row):
    comp = row['Company']
    b_d = row['b_d']
    b_d_s = b_d.strftime('%Y-%m-%d')
    while b_d_s not in prices.index and b_d < pd.Timestamp('2022-03-08'):
        b_d += pd.DateOffset(days=1)
        b_d_s = b_d.strftime('%Y-%m-%d')

    e_d = row['e_d']
    e_d_s = e_d.strftime('%Y-%m-%d')
    while e_d_s not in prices.index and e_d < pd.Timestamp('2022-03-08'):
        e_d += pd.DateOffset(days=1)
        e_d_s = e_d.strftime('%Y-%m-%d')
    e_d = row['b_d']

    if e_d_s not in prices.index:
        return None
        
    b_p = prices[comp][b_d_s]
    e_p = prices[comp][e_d_s]
    return (e_p - b_p) / b_p

In [125]:
articles['p_c'] = articles.apply(cal_price_change, axis=1)

In [127]:
articles.to_csv('./data/price_chg_3m.csv')